# Lacunas Estruturais

Importando bibliotecas:

In [19]:
import pandas as pd
import networkx as nx
import socnet as sn

from random import shuffle
from itertools import combinations

Configurando bibliotecas:

In [20]:
sn.node_size = 5

sn.edge_width = 1
sn.edge_color = (191, 191, 191)

Definindo constantes:

In [21]:
FRAC_CLOSERS = 0.2

NUM_NODES = 25

DENSITY = 0.5

OPENER = 0
CLOSER = 1

OPENER_COLOR = (0, 255, 0)
CLOSER_COLOR = (0, 0, 255)

Exemplo de grafo:

(esse grafo representa o *ambiente* do modelo)

In [22]:
def build_graph():
    g = nx.erdos_renyi_graph(NUM_NODES, DENSITY)

    sn.reset_node_colors(g)
    sn.reset_edge_colors(g)

    return g


g = build_graph()

sn.reset_positions(g)

sn.show_graph(g)

Exemplo de distribuição de tipos de nó:

(esses tipos representam os *agentes* do modelo)

In [23]:
def set_types(g, num_closers=0):
    nodes = list(g.nodes)
    shuffle(nodes)

    for n in nodes[num_closers:]:
        g.nodes[n]['type'] = OPENER

    for n in nodes[:num_closers]:
        g.nodes[n]['type'] = CLOSER

    for n in g.nodes:
        if g.nodes[n]['type'] == OPENER:
            g.nodes[n]['color'] = OPENER_COLOR
        else:
            g.nodes[n]['color'] = CLOSER_COLOR


set_types(g, int(NUM_NODES / 2))

sn.show_graph(g)

Custo de manter uma relação do nó $n$:

$$c_n = \left\{\begin{array}{cl}
0             & \textrm{se }d_n = 0\textrm{;} \\
\frac{1}{d_n} & \textrm{caso contrário,} \\
\end{array}\right.$$

onde $d_n$ é o grau de $n$.

In [24]:
def cost(g, n):
    dn = g.degree(n)

    if dn == 0:
        return 0
    return 1 / dn

Restrição do nó $n$ para *openers* (fórmula de Burt):

$$\sum_{m \in N_n} \left(c_n + \sum_{k \in N_n, k \in N_m} c_n c_k\right)^2\textrm{,}$$

onde $N_n$ é o conjunto de vizinhos de $n$.

Esta fórmula parte da premissa de que o nó não está isolado.

Interpretação informal: o agente busca aumentar suas conexões e prefere conexões indiretas.

In [25]:
def opener_constraint(g, n):
    constraint = 0

    for m in g.neighbors(n):
        sub_constraint = cost(g, n)

        for k in g.neighbors(n):
            if g.has_edge(k, m):
                sub_constraint += cost(g, n) * cost(g, k)

        constraint += sub_constraint**2

    return constraint

Restrição do nó $n$ para *closers* (uma fórmula alternativa):

$$\frac{9}{8} - b_n\textrm{,}$$

onde $b_n$ é a fórmula de Burt.

Esta fórmula parte da premissa de que o nó não está isolado.

Interpretação informal: o agente busca diminuir suas conexões e prefere conexões diretas.

In [26]:
def closer_constraint(g, n):
    indegree = len(list(g.neighbors(n)))
    return 1 - 1/indegree
#     return 9/8 - opener_constraint(g, n)

Restrição geral do nó $n$:

* se o nó estiver isolado, é igual a $2$;
* caso contrário, depende de qual é o tipo.

In [27]:
def constraint(g, n):
    if g.degree(n) == 0:
        return 2

    if g.nodes[n]['type'] == OPENER:
        return opener_constraint(g, n)
    return closer_constraint(g, n)

Inicialização da simulação:

In [28]:
def initialize(g):
    for n in g.nodes:
        g.nodes[n]['constraint'] = constraint(g, n)

Passo da simulação:

1. escolha aleatoriamente um par de nós;
2. considere uma *inversão* da situação desse par, ou seja, remover a aresta se ela existir ou adicionar se não existir;
3. calcule o *ganho* de cada um desses nós, ou seja, a redução da restrição como consequência dessa inversão;
4. no caso particular em que a inversão foi adição, se um dos ganhos for negativo, os dois ganhos são redefinidos como zero *(justificativa no código)*;
5. se um dos ganhos foi positivo aceite a inversão e termine o passo, senão volte para (1);
6. se todos os pares foram escolhidos e nenhuma inversão foi aceita, termine o passo.

In [29]:
def invert(g, n, m):
    if g.has_edge(n, m):
        g.remove_edge(n, m)
    else:
        g.add_edge(n, m)
        g.edges[n, m]['color'] = sn.edge_color


def update(g):
    # Para cada par de nós, em ordem aleatória.

    pairs = list(combinations(g.nodes, 2))
    shuffle(pairs)

    for n, m in pairs:
        # Inverte a situação do par.
        invert(g, n, m)

        # Calcula as restrições após inversão.
        constraint_n = constraint(g, n)
        constraint_m = constraint(g, m)

        # Calcula os ganhos após inversão,
        # (lembrando que restrição é ruim)
        gain_n = g.nodes[n]['constraint'] - constraint_n
        gain_m = g.nodes[m]['constraint'] - constraint_m

        # Se a inversão foi adição e um dos nós foi prejudicado,
        # então o ganho é zero porque essa inversão é impossível.
        # (você não pode forçar uma pessoa a manter uma relação)
        if g.has_edge(n, m) and (gain_n < 0 or gain_m < 0):
            gain_n = 0
            gain_m = 0

        # Se algum nó teve um ganho, aceita a inversão e
        # considera o passo da simulação como terminado,
        # devolvendo True para indicar que houve mudança.
        if gain_n > 0 or gain_m > 0:
            g.nodes[n]['constraint'] = constraint_n
            g.nodes[m]['constraint'] = constraint_m
            return True

        # Se nenhum nó teve um ganho, essa inversão não é aceita.
        # Restauramos, portanto, a situação original do par de nós.
        invert(g, n, m)

    # Se passamos por todos os pares e nenhuma inversão foi aceita,
    # considera o passo da simulação como terminado, devolvendo
    # False para indicar que não houve mudança: a rede estabilizou.
    return False

Simulação com visualização, quando todos os nós buscam reduzir a restrição de Burt:

In [30]:
def simulate(num_closers):
    g = build_graph()

    set_types(g, num_closers)

    initialize(g)

    while update(g):
        pass

    return g


num_closers = 0

g = simulate(num_closers)

sn.reset_positions(g)

sn.show_graph(g)

Inversão da visualização:

In [31]:
def inver_show(g):
    cg = nx.complement(g)

    sn.reset_positions(cg)

    for n in g.nodes:
        g.nodes[n]['pos'] = cg.nodes[n]['pos']

    sn.show_graph(g)


inver_show(g)

Distribuição de *betweenness*:

In [32]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes],
})

df.describe()

,betweenness
count,25.000000
mean,0.020870
std,0.002629
min,0.018395
25%,0.018395
50%,0.018395
75%,0.023551
max,0.023551


Simulação com visualização, quando apenas uma minoria dos nós busca reduzir a restrição de Burt, enquanto todos os outros buscam aumentar:

In [33]:
num_closers = int(FRAC_CLOSERS * NUM_NODES)

g = simulate(num_closers)

sn.reset_positions(g)

sn.show_graph(g)

# Conclusões

O comportamento do nosso Closer foi exatamente o contrário dos Openers. Os openers do nosso grafo desejam ter o máximo de relações possíveis, enquanto que os closers desejam ter apenas relações com pessoas influentes. Por conta disso, os openers formaram um clique, pois estão se relacionando com várias pessoas ao mesmo tempo, enquanto que os closers ficaram na periferia, formando apenas relações "estratégicas" com pessoas influentes.

Isso reflete uma perspectiva do que temos na vida real: as pessoas que desejam ter várias relações (openers) costumam formar um clique (por exemplo políticos), enquanto que as pessoas que desejam ter apenas relações que as beneficiem (closers) ficam na periferia (por exemplo, eleitor).

Distribuição de *betweenness*:

In [34]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes],
})

df.describe()

,betweenness
count,25.000000
mean,0.036812
std,0.043384
min,0.000000
25%,0.019565
50%,0.031159
75%,0.031159
max,0.182609


Distribuição de *betweenness* dos *openers*:

In [35]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes if g.nodes[n]['type'] == OPENER],
})

df.describe()

,betweenness
count,20.000000
mean,0.046014
std,0.043951
min,0.019565
25%,0.019565
50%,0.031159
75%,0.031159
max,0.182609


Distribuição de *betweenness* dos *closers*:

In [36]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes if g.nodes[n]['type'] == CLOSER],
})

df.describe()

,betweenness
count,5.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0
